In [25]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import json
import urllib.request

In [26]:
data = [
    {"nazwa": "DOLNOŚLĄSKIE", "2019": 3, "2020": 2, "2021": 1, "2022": 1, "2023": 2, "2024": 2},
    {"nazwa": "KUJAWSKO-POMORSKIE", "2019": 13, "2020": 11, "2021": 11, "2022": 11, "2023": 12, "2024": 12},
    {"nazwa": "LUBELSKIE", "2019": 14, "2020": 14, "2021": 13, "2022": 15, "2023": 14, "2024": 14},
    {"nazwa": "LUBUSKIE", "2019": 9, "2020": 10, "2021": 9, "2022": 9, "2023": 9, "2024": 9},
    {"nazwa": "ŁÓDZKIE", "2019": 8, "2020": 8, "2021": 7, "2022": 7, "2023": 6, "2024": 6},
    {"nazwa": "MAŁOPOLSKIE", "2019": 4, "2020": 4, "2021": 4, "2022": 5, "2023": 5, "2024": 5},
    {"nazwa": "MAZOWIECKIE", "2019": 1, "2020": 1, "2021": 2, "2022": 2, "2023": 1, "2024": 1},
    {"nazwa": "OPOLSKIE", "2019": 7, "2020": 9, "2021": 10, "2022": 10, "2023": 8, "2024": 8},
    {"nazwa": "PODKARPACKIE", "2019": 12, "2020": 13, "2021": 15, "2022": 13, "2023": 13, "2024": 13},
    {"nazwa": "PODLASKIE", "2019": 11, "2020": 12, "2021": 12, "2022": 12, "2023": 11, "2024": 11},
    {"nazwa": "POMORSKIE", "2019": 5, "2020": 6, "2021": 6, "2022": 6, "2023": 7, "2024": 7},
    {"nazwa": "ŚLĄSKIE", "2019": 2, "2020": 3, "2021": 3, "2022": 3, "2023": 3, "2024": 3},
    {"nazwa": "ŚWIĘTOKRZYSKIE", "2019": 15, "2020": 15, "2021": 14, "2022": 14, "2023": 15, "2024": 15},
    {"nazwa": "WARMIŃSKO-MAZURSKIE", "2019": 16, "2020": 16, "2021": 16, "2022": 16, "2023": 16, "2024": 16},
    {"nazwa": "WIELKOPOLSKIE", "2019": 6, "2020": 5, "2021": 5, "2022": 4, "2023": 4, "2024": 4},
    {"nazwa": "ZACHODNIOPOMORSKIE", "2019": 10, "2020": 7, "2021": 8, "2022": 8, "2023": 10, "2024": 10}
]

In [27]:
df = pd.DataFrame(data)

In [28]:
df_long = df.melt(id_vars = 'nazwa', var_name = 'rok', value_name = 'pozycja')
df_long['rok'] = df_long['rok'].astype(int)

In [30]:
#Wczytywanie województw
geojson_url = "https://raw.githubusercontent.com/ppatrzyk/polska-geojson/master/wojewodztwa/wojewodztwa-medium.geojson"

with urllib.request.urlopen(geojson_url) as response:
    woj = json.load(response)


In [31]:
#Ustalanie klucza
properties_keys = woj["features"][0]["properties"].keys()
print("Klucze properties w GeoJSON:", properties_keys)

geo_name_key = "nazwa"
featureidkey = "properties." + geo_name_key

Klucze properties w GeoJSON: dict_keys(['id', 'nazwa'])


In [32]:
#Mapowanie nazw
geo_names = []
for f in woj["features"]:
    geo_names.append(f["properties"][geo_name_key])

geo_map = {}
for n in geo_names:
    geo_map[str(n).strip().upper()] = n

df_long["geo_nazwa"] = df_long["nazwa"].map(geo_map)

missing = df_long[df_long["geo_nazwa"].isna()]["nazwa"].unique()
print("Brak dopasowania dla (jeśli puste, to OK):", missing)

Brak dopasowania dla (jeśli puste, to OK): []


In [33]:
#Centralki
centroid_rows = []

for feature in woj["features"]:
    region_name = feature["properties"][geo_name_key]
    geometry = feature["geometry"]
    geom_type = geometry["type"]
    coords = geometry["coordinates"]

    points_lon = []
    points_lat = []

    if geom_type == "Polygon":
        outer_ring = coords[0]
        for p in outer_ring:
            points_lon.append(p[0])
            points_lat.append(p[1])

    if geom_type == "MultiPolygon":
        for polygon in coords:
            outer_ring = polygon[0]
            for p in outer_ring:
                points_lon.append(p[0])
                points_lat.append(p[1])

    lon_center = sum(points_lon) / len(points_lon)
    lat_center = sum(points_lat) / len(points_lat)

    centroid_rows.append({
        "geo_nazwa": region_name,
        "lon": lon_center,
        "lat": lat_center
    })

centroids_df = pd.DataFrame(centroid_rows)

In [34]:
#Centralki + dane
df_labels = pd.merge(
    df_long,
    centroids_df,
    on="geo_nazwa",
    how="left"
)

missing_centroids = df_labels[df_labels["lat"].isna()]["geo_nazwa"].unique()
print("Brak centroidów dla (jeśli puste, to OK):", missing_centroids)

Brak centroidów dla (jeśli puste, to OK): []


In [35]:
#wykres 1
min_rank = 1
max_rank = 16

fig = px.choropleth(
    df_long,
    geojson=woj,
    locations="geo_nazwa",
    featureidkey=featureidkey,
    color="pozycja",
    animation_frame="rok",
    hover_name="nazwa",
    hover_data={"pozycja": True, "rok": True, "geo_nazwa": False},
    range_color=[min_rank, max_rank],
    color_continuous_scale="Viridis"
)

fig.update_geos(
    fitbounds="locations",
    visible=False
)

fig.update_layout(
    autosize=True,
    coloraxis_reversescale=True,   # 1 = najlepsze
    title="Ranking województw wg. MRG",
    margin=dict(l=0, r=0, t=60, b=0)
)

Output hidden; open in https://colab.research.google.com to view.

In [36]:
#wykres 2
first_year = int(df_labels["rok"].min())
df_first = df_labels[df_labels["rok"] == first_year]

label_trace = go.Scattergeo(
    lon=df_first["lon"],
    lat=df_first["lat"],
    text=df_first["pozycja"].astype(str),
    mode="text",
    textfont=dict(size=14),
    showlegend=False,
    hoverinfo="skip"
)

fig.add_trace(label_trace)

for frame in fig.frames:
    year = int(frame.name)
    df_year = df_labels[df_labels["rok"] == year]

    frame_label_trace = go.Scattergeo(
        lon=df_year["lon"],
        lat=df_year["lat"],
        text=df_year["pozycja"].astype(str),
        mode="text",
        textfont=dict(size=14),
        showlegend=False,
        hoverinfo="skip"
    )

    frame.data = (frame.data[0], frame_label_trace)

In [37]:
fig.show(config={"responsive": True})

Output hidden; open in https://colab.research.google.com to view.

In [38]:
pio.write_html(
    fig,
    file="mapa_woj_mrg.html",
    include_plotlyjs="cdn",
    auto_open=False,
    config={"responsive": True}
)